In [4]:
import os
from tqdm import tqdm
from glob import glob
import random
import cv2
from PIL import Image
import numpy as np
import albumentations as A
import shutil

In [10]:
def create_train_patched_images(input_folder_path, output_folder_path, patch_size=(224, 224)):
    np.random.seed(42)
    random.seed(42)

    real_image_files = glob(os.path.join(input_folder_path, "0_real_images/*.[pP][nN][gG]"))
    real_image_files += glob(os.path.join(input_folder_path, "0_real_images/*.[jJ][pP][gG]"))

    fake_image_files = glob(os.path.join(input_folder_path, "1_fake_images/*.[pP][nN][gG]"))
    fake_image_files += glob(os.path.join(input_folder_path, "1_fake_images/*.[jJ][pP][gG]"))
    
    # # Randomly sample images
    # sample_size = 10000
    # if len(real_image_files) > sample_size and len(fake_image_files) > sample_size:
    #     real_image_files = random.sample(real_image_files, sample_size)
    #     fake_image_files = random.sample(fake_image_files, sample_size)

    # Processing real images
    print("===========Real Image generating===========")
    os.makedirs(os.path.join(output_folder_path, '0_real_images'), exist_ok=True)
    for image_file in tqdm(real_image_files):
        image = Image.open(image_file)
        w, h = image.size
        output_file_base_name = os.path.basename(image_file).split('.')[0]

        # If the image is smaller than the patch size, resize and save
        if h <= patch_size[1] or w <= patch_size[0]:
            resized_image = image.resize(patch_size)
            output_file_path = os.path.join(output_folder_path, '0_real_images', f'{output_file_base_name}_resized.png')
            resized_image.save(output_file_path)
            continue

        # If the image is larger than the patch size, extract patches
        i = 0
        for top in range(0, h, patch_size[1]):
            for left in range(0, w, patch_size[0]):
                right = min(w, left + patch_size[0])
                bottom = min(h, top + patch_size[1])
                
                if (right - left) == patch_size[0] and (bottom - top) == patch_size[1]:
                    patch = image.crop((left, top, right, bottom))
                    output_file_path = os.path.join(output_folder_path, '0_real_images', f'{output_file_base_name}_patch_{i}.png')
                    patch.save(output_file_path)
                    i += 1

    # Processing fake images
    print("===========Fake Image generating===========")
    os.makedirs(os.path.join(output_folder_path, '1_fake_images'), exist_ok=True)
    for image_file in tqdm(fake_image_files):
        image = Image.open(image_file)
        w, h = image.size
        output_file_base_name = os.path.basename(image_file).split('.')[0]

        # If the image is smaller than the patch size, resize and save
        if h <= patch_size[1] or w <= patch_size[0]:
            resized_image = image.resize(patch_size)
            output_file_path = os.path.join(output_folder_path, '1_fake_images', f'{output_file_base_name}_resized.png')
            resized_image.save(output_file_path)
            continue

        # If the image is larger than the patch size, extract patches
        i = 0
        for top in range(0, h, patch_size[1]):
            for left in range(0, w, patch_size[0]):
                right = min(w, left + patch_size[0])
                bottom = min(h, top + patch_size[1])
                
                if (right - left) == patch_size[0] and (bottom - top) == patch_size[1]:
                    patch = image.crop((left, top, right, bottom))
                    output_file_path = os.path.join(output_folder_path, '1_fake_images', f'{output_file_base_name}_patch_{i}.png')
                    patch.save(output_file_path)
                    i += 1

In [11]:
create_train_patched_images('../../FakeImageDetection_Dataset/train', '../Dataset/patched_train')
create_train_patched_images('../../FakeImageDetection_Dataset/valid', '../Dataset/patched_valid')

===========Real Image generating===========


100%|██████████| 8000/8000 [04:32<00:00, 29.32it/s]


===========Fake Image generating===========


100%|██████████| 8000/8000 [13:09<00:00, 10.13it/s]


===========Real Image generating===========


100%|██████████| 2000/2000 [01:09<00:00, 28.94it/s]


===========Fake Image generating===========


100%|██████████| 2000/2000 [03:20<00:00,  9.96it/s]


In [12]:
def create_test_patched_images(input_image_path, save_path, crop_size=(224, 224), n_patch=5):
    # Image loading
    test_images = glob(f'{input_image_path}/*.png')

    for i in range(n_patch):
        os.makedirs(f'{save_path}/images_{i}', exist_ok=True)

    # Define transforms
    center_crop = A.CenterCrop(*crop_size, p=1.0)
    random_crop = A.RandomCrop(*crop_size, p=1.0)
    resize_transform = A.Resize(*crop_size, p=1.0)

    print("===========Test Image generating===========")
    # Test image transforms
    for img_path in tqdm(test_images):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        h, w, _ = img.shape

        # Check image size and apply appropriate transform
        if h > crop_size[0] and w > crop_size[1]:
            transformed = center_crop(image=img)
            transformed_image = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
            cv2.imwrite(os.path.join(save_path, f'images_0', os.path.basename(img_path)), transformed_image)
            for i in range(1, n_patch):
                np.random.seed(i)
                random.seed(i)
                transformed = random_crop(image=img)
                transformed_image = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
                cv2.imwrite(os.path.join(save_path, f'images_{i}', os.path.basename(img_path)), transformed_image)
        elif h <= crop_size[0] or w <= crop_size[1]:
            for i in range(n_patch):
                transformed = resize_transform(image=img)
                transformed_image = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
                cv2.imwrite(os.path.join(save_path, f'images_{i}', os.path.basename(img_path)), transformed_image)

In [13]:
create_test_patched_images('../../test_data/images', '../Dataset/patched_test')

===========Test Image generating===========


100%|██████████| 58348/58348 [22:04<00:00, 44.04it/s]


In [14]:
def create_train_dct_images(input_image_path, save_path):
    real_image_files = glob(os.path.join(input_image_path, "0_real_images/*.[pP][nN][gG]")) 
    fake_image_files = glob(os.path.join(input_image_path, "1_fake_images/*.[pP][nN][gG]"))

    # Define subfolders for saving DCT images
    real_dct_folder = os.path.join(save_path, '0_real_images')
    fake_dct_folder = os.path.join(save_path, '1_fake_images')
    
    os.makedirs(real_dct_folder, exist_ok=True)
    os.makedirs(fake_dct_folder, exist_ok=True)
    
    # Processing real images
    print("===========Train Real Image DCT generating===========")
    for image_file in tqdm(real_image_files):
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Perform DCT on each channel of the image
        dct = np.zeros_like(image, dtype=np.float32)
        for channel in range(3):
            dct[:, :, channel] = cv2.dct(np.float32(image[:, :, channel]))/255.0
            
        output_file_base_name = os.path.basename(image_file).split('.')[0]
        output_file_path = os.path.join(real_dct_folder, f'{output_file_base_name}_dct.png')
        cv2.imwrite(output_file_path, dct * 255)  # Save the DCT image

    # Processing fake images
    print("===========Train Fake Image DCT generating===========")
    for image_file in tqdm(fake_image_files):
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Perform DCT on each channel of the image
        dct = np.zeros_like(image, dtype=np.float32)
        for channel in range(3):
            dct[:, :, channel] = cv2.dct(np.float32(image[:, :, channel]))/255.0
            
        output_file_base_name = os.path.basename(image_file).split('.')[0]
        output_file_path = os.path.join(fake_dct_folder, f'{output_file_base_name}_dct.png')
        cv2.imwrite(output_file_path, dct * 255)  # Save the DCT image

In [15]:
def create_test_dct_images(input_folder_path, save_folder_path, n_patch):
    for i in range(n_patch):
        # Get image files
        image_files = glob(os.path.join(input_folder_path, f"images_{i}/*.[pP][nN][gG]"))
        # Create save directory if not exists
        save_dir = os.path.join(save_folder_path, f'images_{i}')
        os.makedirs(save_dir, exist_ok=True)
        
        # Processing images
        print(f"===========Test DCT Image generating for patch {i+1}/{n_patch}===========")
        for image_file in tqdm(image_files):
            # Load and convert the image
            image = cv2.imread(image_file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Perform DCT on each channel of the image
            dct = np.zeros_like(image, dtype=np.float32)
            for channel in range(3):
                dct[:, :, channel] = cv2.dct(np.float32(image[:, :, channel])/255.0)
            # Save the DCT transformed image
            output_file_base_name = os.path.basename(image_file).split('.')[0]
            output_file_path = os.path.join(save_dir, f'{output_file_base_name}.png')
            cv2.imwrite(output_file_path, dct * 255)  # Save the DCT image

In [16]:
create_train_dct_images('../Dataset/patched_train', '../Dataset/dct_train')
create_train_dct_images('../Dataset/patched_valid', '../Dataset/dct_valid')

===========Train Real Image DCT generating===========


100%|██████████| 16349/16349 [01:28<00:00, 184.08it/s]


===========Train Fake Image DCT generating===========


100%|██████████| 44255/44255 [04:45<00:00, 154.84it/s]


===========Train Real Image DCT generating===========


100%|██████████| 4071/4071 [00:27<00:00, 150.25it/s]


===========Train Fake Image DCT generating===========


100%|██████████| 11124/11124 [01:17<00:00, 144.25it/s]


In [17]:
create_test_dct_images('../Dataset/patched_test', '../Dataset/dct_test', 5)

===========Test DCT Image generating for patch 1/5===========


100%|██████████| 58348/58348 [06:25<00:00, 151.35it/s]


===========Test DCT Image generating for patch 2/5===========


100%|██████████| 58348/58348 [06:35<00:00, 147.38it/s]


===========Test DCT Image generating for patch 3/5===========


100%|██████████| 58348/58348 [06:25<00:00, 151.46it/s]


===========Test DCT Image generating for patch 4/5===========


100%|██████████| 58348/58348 [06:33<00:00, 148.44it/s]


===========Test DCT Image generating for patch 5/5===========


100%|██████████| 58348/58348 [06:29<00:00, 149.71it/s]


In [2]:
def create_train_fft_images(input_image_path, save_path):
    real_image_files = glob(os.path.join(input_image_path, "0_real_images/*.[pP][nN][gG]")) 
    fake_image_files = glob(os.path.join(input_image_path, "1_fake_images/*.[pP][nN][gG]"))

    # Define subfolders for saving FFT images
    real_fft_folder = os.path.join(save_path, '0_real_images')
    fake_fft_folder = os.path.join(save_path, '1_fake_images')
    
    os.makedirs(real_fft_folder, exist_ok=True)
    os.makedirs(fake_fft_folder, exist_ok=True)
    
    # Processing real images
    print("===========Train Real Image FFT generating===========")
    for image_file in tqdm(real_image_files):
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Perform FFT on each channel of the image
        fft = np.zeros_like(image, dtype=np.float32)
        for channel in range(3):
            fft_channel = np.fft.fft2(image[:, :, channel])
            fft_channel_magnitude = np.abs(fft_channel)
            fft[:, :, channel] = np.log(fft_channel_magnitude + 1)  # use log scale for better visualization

        output_file_base_name = os.path.basename(image_file).split('.')[0]
        output_file_path = os.path.join(real_fft_folder, f'{output_file_base_name}_fft.png')
        cv2.imwrite(output_file_path, cv2.normalize(fft, None, 0, 255, cv2.NORM_MINMAX))  # Save the FFT image

    # Processing fake images
    print("===========Train Fake Image FFT generating===========")
    for image_file in tqdm(fake_image_files):
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Perform FFT on each channel of the image
        fft = np.zeros_like(image, dtype=np.float32)
        for channel in range(3):
            fft_channel = np.fft.fft2(image[:, :, channel])
            fft_channel_magnitude = np.abs(fft_channel)
            fft[:, :, channel] = np.log(fft_channel_magnitude + 1)  # use log scale for better visualization

        output_file_base_name = os.path.basename(image_file).split('.')[0]
        output_file_path = os.path.join(fake_fft_folder, f'{output_file_base_name}_fft.png')
        cv2.imwrite(output_file_path, cv2.normalize(fft, None, 0, 255, cv2.NORM_MINMAX))  # Save the FFT image

In [3]:
def create_test_fft_images(input_folder_path, save_folder_path, n_patch):
    for i in range(n_patch):
        # Get image files
        image_files = glob(os.path.join(input_folder_path, f"images_{i}/*.[pP][nN][gG]"))
        # Create save directory if not exists
        save_dir = os.path.join(save_folder_path, f'images_{i}')
        os.makedirs(save_dir, exist_ok=True)
        
        # Processing images
        print(f"===========Test FFT Image generating for patch {i+1}/{n_patch}===========")
        for image_file in tqdm(image_files):
            image = cv2.imread(image_file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Perform FFT on each channel of the image
            fft = np.zeros_like(image, dtype=np.float32)
            for channel in range(3):
                fft_channel = np.fft.fft2(image[:, :, channel])
                fft_channel_magnitude = np.abs(fft_channel)
                fft[:, :, channel] = np.log(fft_channel_magnitude + 1)  # use log scale for better visualization

            output_file_base_name = os.path.basename(image_file).split('.')[0]
            output_file_path = os.path.join(save_dir, f'{output_file_base_name}.png')
            cv2.imwrite(output_file_path, cv2.normalize(fft, None, 0, 255, cv2.NORM_MINMAX))  # Save the FFT image

In [5]:
create_train_fft_images('../Dataset/patched_train', '../Dataset/fft_train')
create_train_fft_images('../Dataset/patched_valid', '../Dataset/fft_valid')

===========Train Real Image FFT generating===========


100%|██████████| 16349/16349 [02:09<00:00, 126.68it/s]


===========Train Fake Image FFT generating===========


100%|██████████| 44255/44255 [06:07<00:00, 120.55it/s]


===========Train Real Image FFT generating===========


100%|██████████| 4071/4071 [00:34<00:00, 117.08it/s]


===========Train Fake Image FFT generating===========


100%|██████████| 11124/11124 [01:34<00:00, 117.47it/s]


In [6]:
create_test_fft_images('../Dataset/patched_test', '../Dataset/fft_test', 5)

===========Test FFT Image generating for patch 1/5===========


100%|██████████| 58348/58348 [08:33<00:00, 113.62it/s]


===========Test FFT Image generating for patch 2/5===========


100%|██████████| 58348/58348 [08:54<00:00, 109.19it/s]


===========Test FFT Image generating for patch 3/5===========


100%|██████████| 58348/58348 [08:53<00:00, 109.42it/s]


===========Test FFT Image generating for patch 4/5===========


100%|██████████| 58348/58348 [08:52<00:00, 109.51it/s]


===========Test FFT Image generating for patch 5/5===========


100%|██████████| 58348/58348 [09:07<00:00, 106.50it/s]
